In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [13]:
stocks = pd.read_csv('Data_clean/6_Portfolios_ME_Prior_12_2_returns.csv')
bonds = pd.read_csv('Data_clean/bond_returns.csv')
riskfree = pd.read_csv('Data_clean/FF_cleaned.csv')

stocks = stocks[['Date', 'Market Return']]
stocks = stocks.rename(columns={'Market Return': 'Stock Return'})
stocks.set_index('Date', inplace=True)
stocks['Stock Return'] = stocks['Stock Return'] / 100

bonds = bonds[['Date', '10YrReturns']]
bonds = bonds.rename(columns={'10YrReturns': 'Bond Return'})
bonds.set_index('Date', inplace=True)

riskfree = riskfree[['Date', 'RF']]
riskfree.set_index('Date', inplace=True)
riskfree['RF'] = riskfree['RF'] / 100

data_all = pd.concat([stocks, bonds, riskfree], axis=1)
data = data_all[data_all.index >= '1990-01-31']
data_minus_rf = data[['Stock Return', 'Bond Return']]

In [14]:
target_return = 0.0075
mu = data_minus_rf.mean()
sigma = data_minus_rf.cov()

In [15]:
mu

Stock Return    0.009310
Bond Return     0.004323
dtype: float64

In [16]:
sigma

,Stock Return,Bond Return
Stock Return,0.001945,-0.000064
Bond Return,-0.000064,0.000452


Markowitz (unleveraged - closed solution):

In [4]:
sigma_inv = np.linalg.inv(sigma)
ones = np.ones(len(mu))
a = mu.transpose() @ sigma_inv @ mu
b = mu.transpose() @ sigma_inv @ ones
c = ones.transpose() @ sigma_inv @ ones
A = np.array([[a, b], [b, c]])
A_inv = np.linalg.inv(A)
e = np.array([target_return, 1])

In [5]:
weights_ma_u = sigma_inv @ np.array([mu,ones]).T @ A_inv @ np.array([target_return, 1])
exp_return_ma_u = weights_ma_u @ mu
risk_ma_u = np.sqrt(weights_ma_u @ sigma @ weights_ma_u)
print('Markowitz (unleveraged): ','Bonds', weights_ma_u[1].round(3), 'Stocks', weights_ma_u[0].round(3))
print('Expected return:', (exp_return_ma_u).round(4))
print('std:', (risk_ma_u).round(4))

Markowitz (unleveraged):  Bonds 0.363 Stocks 0.637
Expected return: 0.0075
std: 0.0286


Markowitz (unleveraged - scipy optimization):


In [6]:
def objective(weights, sigma):
    return 0.5 * weights @ sigma @ weights

constraints = (
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}, 
    {'type': 'eq', 'fun': lambda weights: np.dot(weights, mu) - target_return}  
)
bounds = tuple((0, 1) for asset in range(len(mu)))

In [7]:
initial_weights = np.array([0.1,0.9])
result = minimize(objective, initial_weights, args=(sigma), method='SLSQP', bounds=bounds, constraints=constraints)
print("Markowitz (unleveraged):", result.x.round(3))
print("Expected return:", np.dot(result.x, mu).round(4))
print("std:", np.sqrt(objective(result.x, sigma)).round(4))

Markowitz (unleveraged): [0.637 0.363]
Expected return: 0.0075
std: 0.0202


60/40 strategy:

In [8]:
weights_60_40 = np.array([0.6, 0.4])
exp_return_60_40 = np.dot(weights_60_40, mu)
risk_60_40 = np.sqrt(objective(weights_60_40, sigma))
print('60/40:', 'Stocks', weights_60_40[0], 'Bonds', weights_60_40[1])
print('Expected return:', exp_return_60_40.round(4))
print('std:', risk_60_40.round(4))

60/40: Stocks 0.6 Bonds 0.4
Expected return: 0.0073
std: 0.0193


Markowitz (leveraged - closed solution):

In [9]:
data_excess = data_minus_rf - data['RF'].values.reshape(-1,1)
mu_excess = data_excess.mean()
sigma_excess = data_excess.cov()
RF = data['RF'].mean()

In [10]:
RF = data['RF'].mean()
mu_excess_target = 0.0075 - RF

In [11]:
sigma_inv_excess = np.linalg.inv(sigma_excess)
B = mu_excess.transpose() @ sigma_inv_excess @ mu_excess
C = sigma_inv_excess @ mu_excess

weights_ma_l = mu_excess_target * (C / B)

In [12]:
exp_return_ma_l = np.dot(weights_ma_l, mu_excess) + RF
exp_excess_return_ma_l = np.dot(weights_ma_l, mu_excess)
risk_ma_l = np.sqrt(weights_ma_l @ sigma_excess @ weights_ma_l)

print('Markowitz (leveraged):', 'Bonds', weights_ma_l[1].round(3), 'Stocks', weights_ma_l[0].round(3), 'Risk free', (1-weights_ma_l.sum()).round(3))
print('Expected return:', exp_return_ma_l.round(4))
print('std:', risk_ma_l.round(4))
print('Risk free mean return:', RF.round(4))

Markowitz (leveraged): Bonds 0.739 Stocks 0.523 Risk free -0.261
Expected return: 0.0075
std: 0.0269
Risk free mean return: 0.0021


Markowitz (leveraged - scipy optimization):

In [13]:
def objective_l(weights, sigma):
    return 0.5 * weights @ sigma @ weights

constraints_l = (
    {'type': 'eq', 'fun': lambda weights: np.dot(weights, mu_excess) - mu_excess_target}#,
    #{'type': 'ineq', 'fun': lambda weights: 1.5 - np.sum(weights)}  
)
initial_weights_l = np.array([0.3,0.3,0.2])
bounds_l = [(0, None) for _ in range(len(initial_weights))]
result_l = minimize(objective_l, initial_weights_l, args=(sigma_excess), method='SLSQP', bounds=bounds_l, constraints=constraints_l)

print("Markowitz (leveraged):", result_l.x.round(3))
print("Expected excess return:", np.dot(result_l.x, mu).round(4))
print("std:", np.sqrt(objective(result_l.x, sigma)).round(4))

Markowitz (leveraged): [0.587 0.527]
Expected excess return: 0.0077
std: 0.0194


In [14]:
weight_rf = 1 - np.sum(result.x)
print('Risk free:', weight_rf.round(3))

Risk free: 0.0


Test ---

In [ ]:
mu_test = data.mean()
sigma_test = data.cov()
mu_target = 0.0075

In [34]:
def objective_test(weights, sigma):
    return 0.5 * weights @ sigma @ weights

constraints_test = (
    {'type': 'eq', 'fun': lambda weights: np.dot(weights, mu_test) - mu_target},
    {'type': 'eq', 'fun': lambda weights: 1 - np.sum(weights)}  
)
initial_weights_test = np.array([0.3,0.3,0.2])
bounds_test = [(0, None) if i < 2 else (None, None) for i in range(len(initial_weights_test))]
#bounds_test = [(0, None) for _ in range(len(initial_weights_test))]
result_test = minimize(objective_test, initial_weights_test, args=(sigma_test), method='SLSQP', bounds=bounds_test, constraints=constraints_test)

print("Markowitz (leveraged):", result_test.x.round(3))
print("Expected excess return:", np.dot(result_test.x, mu_test).round(4))
print("std:", np.sqrt(objective_test(result_test.x, sigma_test)).round(4))

Markowitz (leveraged): [0.656 0.3   0.044]
Expected excess return: 0.0075
std: 0.0207


Test ---

Risk parity:

In [15]:
def risk_parity_fun(w,sigma):
    std = np.sqrt(w @ sigma @ w)
    N  = len(w)
    w_rp = std**2 / ((sigma @ w) * N)
    objective_rp = np.sum((w - w_rp)**2)
    
    return objective_rp

In [16]:
constraints_rp = ({
    'type': 'ineq',
    'fun': lambda weights: 1.5 - np.sum(weights)  # This should sum weights to exactly 1
})


bounds_rp = [(0, None) for _ in range(len(initial_weights))]

initial_weights_rp = np.array([0.5, 0.5])
result_rp = minimize(risk_parity_fun, initial_weights_rp, args=(sigma_excess), method='SLSQP', bounds=bounds_rp, constraints=constraints_rp)

print("Risk Parity Portfolio Weights:", result_rp.x.round(3))
print("Expected excess return:", np.dot(result_rp.x, mu_excess).round(4))
print("std:", np.sqrt(objective(result_rp.x, sigma_excess)).round(4))
print("risk contribution per asset:", ((result_rp.x[0] * (sigma @ result_rp.x).iloc[0])/np.sqrt(objective(result_rp.x, sigma))).round(4), 
      ((result_rp.x[1] * (sigma @ result_rp.x).iloc[1])/np.sqrt(objective(result_rp.x, sigma))).round(4))

Risk Parity Portfolio Weights: [0.157 0.329]
Expected excess return: 0.0018
std: 0.0067
risk contribution per asset: 0.0067 0.0068


In [17]:
k = mu_excess_target / (mu_excess @ result_rp.x)
weights_rp = k * result_rp.x

exp_return_rp = np.dot(weights_rp, mu_excess) + RF
exp_excess_return_rp = np.dot(weights_rp, mu_excess)
risk_rp = np.sqrt(weights_rp @ sigma_excess @ weights_rp)

print('Risk Parity Portfolio Weights (leveraged):', weights_rp.round(3))
print('Expected return:', exp_return_rp.round(4))
print('std:', risk_rp.round(4))
print('Risk free weight:', (1-weights_rp.sum()).round(3))

Risk Parity Portfolio Weights (leveraged): [0.457 0.955]
Expected return: 0.0075
std: 0.0275
Risk free weight: -0.411


In [18]:
plot_data = pd.DataFrame({
    'Markowitz (unleveraged)': np.append(weights_ma_u[::-1], [0, exp_return_ma_u, risk_ma_u]),
    'Markowitz (leveraged)': np.append(weights_ma_l, [1 - weights_ma_l.sum(), exp_return_ma_l, risk_ma_l]), 
    '60/40': np.append(weights_60_40[::-1], [0, exp_return_60_40, risk_60_40]),  
    'Risk Parity': np.append(weights_rp,[1-weights_rp.sum(), exp_return_rp, risk_rp])  
}, index=['Bonds', 'Stocks', 'RF', 'Expected Return', 'std'])  
plot_data = plot_data.round(4)

In [19]:
plot_data

,Markowitz (unleveraged),Markowitz (leveraged),60/40,Risk Parity
Bonds,0.3629,0.5226,0.4000,0.4568
Stocks,0.6371,0.7388,0.6000,0.9546
RF,0.0000,-0.2614,0.0000,-0.4115
Expected Return,0.0075,0.0075,0.0073,0.0075
std,0.0286,0.0269,0.0193,0.0275
